*참고자료*
- https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/
- https://towardsdatascience.com/understanding-lightgbm-parameters-and-how-to-tune-them-6764e20c6e5b

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb


# 시각화
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
sns.set(font="THEGaeideuk",rc={"axes.unicode_minus":False},style='white')

## 데이터 로드

In [2]:
err=pd.read_csv('./data/train_err_data.csv')
quality=pd.read_csv('./data/train_quality_data.csv')
problem=pd.read_csv('./data/train_problem_data.csv')
test_err=pd.read_csv('./data/test_err_data.csv')
test_quality=pd.read_csv('./data/test_quality_data.csv')

C:\Users\oye01\anaconda3\envs\pydatavenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\oye01\anaconda3\envs\pydatavenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
quality

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [4]:
test_quality

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,2,0,0,0,0,3,0,0
747968,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
747969,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
747970,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0


In [23]:
err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


In [24]:
test_err

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0
16532644,44998,20201130211831,model_1,04.16.3553,31,1
16532645,44998,20201130211832,model_1,04.16.3553,15,1
16532646,44998,20201130212259,model_1,04.16.3553,16,1


In [5]:
# data 타입 맞추기
err['errtype']=err['errtype'].astype('str')
test_err['errtype']=test_err['errtype'].astype('int')
test_err['errtype']=test_err['errtype'].astype('str')

In [6]:
# train, test set 만들기
train=[i for i in range(10000,25000)]
test=[i for i in range(30000,44999)]
train=pd.DataFrame(train,columns=['user_id'])
test=pd.DataFrame(test,columns=['user_id'])

In [5]:
err.isna().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     1
dtype: int64

In [6]:
test_err.isna().sum()

user_id     0
time        0
model_nm    0
fwver       0
errtype     0
errcode     4
dtype: int64

In [7]:
err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   object
 5   errcode   object
dtypes: int64(2), object(4)
memory usage: 757.8+ MB


In [8]:
test_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16532648 entries, 0 to 16532647
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   object
 5   errcode   object
dtypes: int64(2), object(4)
memory usage: 756.8+ MB


In [9]:
err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


In [10]:
test_err

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0
16532644,44998,20201130211831,model_1,04.16.3553,31,1
16532645,44998,20201130211832,model_1,04.16.3553,15,1
16532646,44998,20201130212259,model_1,04.16.3553,16,1


# 모델링 (하나씩)

# err 모델링, tuple(model_nm , fwver) 

In [7]:
#더미 컬럼 만들기, user id 기준
model_col=pd.get_dummies(err[['user_id','model_nm']],columns=['model_nm']).groupby(['user_id']).sum()
print(model_col)

         model_nm_model_0  model_nm_model_1  model_nm_model_2  \
user_id                                                         
10000                 0.0               0.0               0.0   
10001                 0.0               0.0            2365.0   
10002                 0.0               0.0               0.0   
10003                 0.0               0.0             306.0   
10004               777.0               0.0               0.0   
...                   ...               ...               ...   
24995                 0.0               0.0             194.0   
24996                 0.0               0.0               0.0   
24997               826.0               0.0               0.0   
24998               155.0               0.0               0.0   
24999                 0.0               0.0               0.0   

         model_nm_model_3  model_nm_model_4  model_nm_model_5  \
user_id                                                         
10000               317.

In [8]:
model_col=(model_col>0).reset_index()
print(model_col)

       user_id  model_nm_model_0  model_nm_model_1  model_nm_model_2  \
0        10000             False             False             False   
1        10001             False             False              True   
2        10002             False             False             False   
3        10003             False             False              True   
4        10004              True             False             False   
...        ...               ...               ...               ...   
14995    24995             False             False              True   
14996    24996             False             False             False   
14997    24997              True             False             False   
14998    24998              True             False             False   
14999    24999             False             False             False   

       model_nm_model_3  model_nm_model_4  model_nm_model_5  model_nm_model_6  \
0                  True             False             

In [9]:
train

,user_id
0,10000
1,10001
2,10002
3,10003
4,10004
...,...
14995,24995
14996,24996
14997,24997
14998,24998


In [10]:
train=pd.merge(train,model_col)

In [11]:
train

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8
0,10000,False,False,False,True,False,False,False,False,False
1,10001,False,False,True,False,False,False,False,False,False
2,10002,False,False,False,True,False,False,False,False,False
3,10003,False,False,True,False,False,False,False,False,False
4,10004,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
14995,24995,False,False,True,False,False,False,False,False,False
14996,24996,False,False,False,True,False,False,False,False,False
14997,24997,True,False,False,False,False,False,False,False,False
14998,24998,True,False,False,False,False,False,False,False,False


## 튜플 결과 X

In [ ]:
err_tuple=err

In [49]:
err_tuple["tup"]=((tuple(i , j)) for i in err_tuple["model_nm"] for j in err_tuple["fwver"] )

In [50]:
err_tuple

,user_id,time,model_nm,fwver,errtype,errcode,tu,tup
0,10000,20201101025616,model_3,05.15.2138,15,1,0,<generator object <genexpr> at 0x000001CA9437D...
1,10000,20201101030309,model_3,05.15.2138,12,1,0,<generator object <genexpr> at 0x000001CA9437D...
2,10000,20201101030309,model_3,05.15.2138,11,1,0,<generator object <genexpr> at 0x000001CA9437D...
3,10000,20201101050514,model_3,05.15.2138,16,1,0,<generator object <genexpr> at 0x000001CA9437D...
4,10000,20201101050515,model_3,05.15.2138,4,0,0,<generator object <genexpr> at 0x000001CA9437D...
...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1,0,<generator object <genexpr> at 0x000001CA9437D...
16554659,24999,20201130172625,model_3,05.15.2138,16,1,0,<generator object <genexpr> at 0x000001CA9437D...
16554660,24999,20201130172625,model_3,05.15.2138,4,0,0,<generator object <genexpr> at 0x000001CA9437D...
16554661,24999,20201130172631,model_3,05.15.2138,4,0,0,<generator object <genexpr> at 0x000001CA9437D...


## 튜플 XX

In [18]:
def tup(x,y):
    zipped=zip(x,y)
    #a=(x for x,y in zip(err['model_nm'],err['fwver']))
    return tuple(zipped)

In [ ]:
#안돌아감
err_tuple['tu']=err_tuple.apply(lambda x :tup(x['model_nm'],x['fwver']),axis=1)

In [ ]:
#ValueError: cannot set using a multi-index selection indexer with a different length than the value
for i in range(len(err_tuple.index)):
    zipped=zip(err_tuple.iloc[i,2], err_tuple.iloc[i,3])
    err_tuple.iloc[i,6]=tuple(zipped)

In [ ]:
for i in range(len(err_tuple['tu'])):
    zipped=zip(x,y)
    #a=(x for x,y in zip(err['model_nm'],err['fwver']))
    return tuple(zipped)
    zip()

### 튜플 ~

In [52]:
fwv=err_tuple['fwver']
mnm=err_tuple['model_nm']
uid=err_tuple['user_id']
mer=pd.DataFrame([uid,fwv,mnm])

KeyboardInterrupt: 

In [ ]:
#id index처리 
mer.index=mer['user_id']
mer['merged']=[', '.join(sorted(filter(None, x))) for x in df.to_numpy()]
#https://www.python2.net/questions-1109266.htm

In [ ]:
#빠꾸
mer.reset_index( )
mer=mer.get_dummies(mer[['user_id','md_fw']],columns=['merged']).groupby(['user_id']).sum()
mer=(mer>0).reset_index()
train=pd.merge(train,mer)

<br><br>

## Feature Engineering

### model컬럼

In [51]:
# train
model_col=pd.get_dummies(err[['user_id','model_nm']],columns=['model_nm']).groupby(['user_id']).sum()
model_col=(model_col>0).reset_index()
train=pd.merge(train,model_col)

# test
test_err.loc[~test_err['model_nm'].isin(err['model_nm'].unique()),'model_nm']=np.nan
model_col=pd.get_dummies(test_err[['user_id','model_nm']],columns=['model_nm']).groupby(['user_id']).sum()
model_col=(model_col>0).reset_index()
test=pd.merge(test,model_col,how='left',on='user_id')

# train 튜플 XXX (model_nm, fwver)

zip

In [34]:
err_tuple['modelfw']=err_tuple[['model_nm'],['fwver']].apply(lambda x,y: (zip(err['model_nm'],err['fwver']))

SyntaxError: unexpected EOF while parsing (<ipython-input-34-5129b1cda5ee>, line 1)

tup 함수

In [60]:
def tup(x,y):
    zipped=zip(x,y)
    #a=(x for x,y in zip(err['model_nm'],err['fwver']))
    return tuple(zipped)
print(tup([1,2,3],[4,5,6]))

((1, 4), (2, 5), (3, 6))


In [ ]:
err_tuple['modelfw']=err_tuple.apply(lambda x :tup(x['model_nm'],x['fwver']),axis=1)

In [51]:
err_tuple

,user_id,time,model_nm,fwver,errtype,errcode,modelfw
0,10000,20201101025616,model_3,05.15.2138,15,1,0
1,10000,20201101030309,model_3,05.15.2138,12,1,0
2,10000,20201101030309,model_3,05.15.2138,11,1,0
3,10000,20201101050514,model_3,05.15.2138,16,1,0
4,10000,20201101050515,model_3,05.15.2138,4,0,0
...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1,0
16554659,24999,20201130172625,model_3,05.15.2138,16,1,0
16554660,24999,20201130172625,model_3,05.15.2138,4,0,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0,0


# 튜플 개수, 종류

In [ ]:
# Length of values (37) does not match length of index (16554663)
err_tuple['modelfw']={(x,y) for x, y in zip(err['model_nm'],err['fwver'])}

In [30]:
model_tuple={(x,y) for x, y in zip(err['model_nm'],err['fwver'])}

In [31]:
model_tuple

{('model_0', '04.22.1442'),
 ('model_0', '04.22.1656'),
 ('model_0', '04.22.1666'),
 ('model_0', '04.22.1684'),
 ('model_0', '04.22.1750'),
 ('model_0', '04.22.1778'),
 ('model_1', '04.16.2641'),
 ('model_1', '04.16.3345'),
 ('model_1', '04.16.3439'),
 ('model_1', '04.16.3553'),
 ('model_1', '04.16.3569'),
 ('model_1', '04.16.3571'),
 ('model_2', '04.33.1095'),
 ('model_2', '04.33.1125'),
 ('model_2', '04.33.1149'),
 ('model_2', '04.33.1171'),
 ('model_2', '04.33.1185'),
 ('model_2', '04.33.1261'),
 ('model_3', '05.15.2090'),
 ('model_3', '05.15.2092'),
 ('model_3', '05.15.2114'),
 ('model_3', '05.15.2120'),
 ('model_3', '05.15.2122'),
 ('model_3', '05.15.2138'),
 ('model_3', '05.15.3104'),
 ('model_4', '03.11.1141'),
 ('model_4', '03.11.1149'),
 ('model_4', '03.11.1167'),
 ('model_5', '04.82.1684'),
 ('model_5', '04.82.1730'),
 ('model_5', '04.82.1778'),
 ('model_6', '10'),
 ('model_6', '8.5.3'),
 ('model_7', '05.66.3237'),
 ('model_7', '05.66.3571'),
 ('model_8', '04.73.2237'),
 ('mo

In [27]:
model_col

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8
0,30000,False,True,True,False,False,False,False,False,False
1,30001,False,False,False,True,False,False,False,False,False
2,30002,True,False,False,False,False,False,False,False,False
3,30003,True,False,False,False,False,False,False,False,False
4,30004,False,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
14993,44994,False,True,False,False,False,False,False,False,False
14994,44995,True,False,False,False,False,False,False,False,False
14995,44996,True,False,False,False,False,False,False,False,False
14996,44997,True,False,False,False,False,False,False,False,False


In [28]:
train

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errcode_94,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_etc,errcode_standby
0,10000,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,10001,False,False,True,False,False,False,False,False,False,...,False,False,True,True,False,True,False,False,True,True
2,10002,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,10003,False,False,True,False,False,False,False,False,False,...,False,False,True,True,True,True,False,True,True,False
4,10004,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,False,False,True,False,False,False,False,False,False,...,False,False,True,True,False,True,False,False,True,True
14996,24996,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
14997,24997,True,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,True,True,True
14998,24998,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,True,False


할일 
#기존 err 모델에 튜플 열 생성 하기
#quality 모델에 모든 quality 값을 원핫인코딩해서 
#fwver(튜플)별 quality 값 추가 하기 
 err/test_err에 (model,fwver) 튜플 열 넣고, 겟더미 groupby 튜플 열 , merge

# MODELING QUALITY (성희)

## quality 평균

In [62]:
quality

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [66]:
qual_model=quality

In [67]:
qual_model=qual_model.groupby(['user_id','fwver']).mean()

In [68]:
qual_model

,,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
user_id,fwver,,,,,,,,,
10000,05.15.2138,2.020113e+13,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
10002,05.15.2138,2.020112e+13,0.000000,-0.020833,-0.010417,0.0,0.0,0.458333,-0.020833,0.0
10004,04.22.1750,2.020110e+13,-0.083333,-0.083333,-0.083333,0.0,0.0,3.541667,-0.083333,0.0
10005,04.22.1750,2.020112e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,1.083333,-0.416667,0.0
10006,04.16.3553,2.020111e+13,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
24992,04.22.1750,2.020111e+13,-0.333333,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0
24993,05.15.2138,2.020112e+13,-0.291667,-0.291667,-0.291667,0.0,0.0,-0.291667,-0.291667,0.0
24995,04.33.1261,2.020113e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,-0.416667,-0.416667,0.0


In [69]:
qual_model.reset_index(inplace=True)

In [70]:
qual_model

,user_id,fwver,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
0,10000,05.15.2138,2.020113e+13,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,10002,05.15.2138,2.020112e+13,0.000000,-0.020833,-0.010417,0.0,0.0,0.458333,-0.020833,0.0
2,10004,04.22.1750,2.020110e+13,-0.083333,-0.083333,-0.083333,0.0,0.0,3.541667,-0.083333,0.0
3,10005,04.22.1750,2.020112e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,1.083333,-0.416667,0.0
4,10006,04.16.3553,2.020111e+13,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...
10080,24992,04.22.1750,2.020111e+13,-0.333333,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0
10081,24993,05.15.2138,2.020112e+13,-0.291667,-0.291667,-0.291667,0.0,0.0,-0.291667,-0.291667,0.0
10082,24995,04.33.1261,2.020113e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,-0.416667,-0.416667,0.0
10083,24997,04.22.1750,2.020111e+13,-0.333333,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0


In [71]:
qual_model['fwver'].unique()

array(['05.15.2138', '04.22.1750', '04.16.3553', '04.33.1261',
       '04.22.1778', '04.33.1185', '04.16.3571', '05.66.3571',
       '03.11.1149', '03.11.1167', '04.82.1684', '04.82.1778',
       '04.33.1149', '05.66.3237', '04.73.2237', '09.17.1431',
       '04.22.1684', '05.15.2120', '04.33.1125', '05.15.2122',
       '04.22.1666', '04.22.1656', '04.16.3439', '04.73.2571',
       '05.15.2114', '04.16.3345', '04.22.1442'], dtype=object)

#### fwver 두개이상인거 평균

In [77]:
qual_model_1=qual_model.groupby(['user_id']).mean()

In [79]:
qual_model_1.reset_index()

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
0,10000,2.020113e+13,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
1,10002,2.020112e+13,0.000000,-0.020833,-0.010417,0.0,0.0,0.458333,-0.020833,0.0
2,10004,2.020110e+13,-0.083333,-0.083333,-0.083333,0.0,0.0,3.541667,-0.083333,0.0
3,10005,2.020112e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,1.083333,-0.416667,0.0
4,10006,2.020112e+13,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
8219,24990,2.020111e+13,NaN,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0
8220,24992,2.020111e+13,-0.333333,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0
8221,24993,2.020112e+13,-0.291667,-0.291667,-0.291667,0.0,0.0,-0.291667,-0.291667,0.0
8222,24995,2.020113e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,-0.416667,-0.416667,0.0


In [72]:
qual_model_2=pd.get_dummies(qual_model[['user_id','fwver',]],columns=['fwver']).groupby(['user_id']).sum()
qual_model_2=(qual_model_2>0).reset_index()


In [76]:
qual_model_2

,user_id,fwver_03.11.1149,fwver_03.11.1167,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3553,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,...,fwver_04.73.2571,fwver_04.82.1684,fwver_04.82.1778,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.2138,fwver_05.66.3237,fwver_05.66.3571,fwver_09.17.1431
0,10000,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,10002,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,10004,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,10005,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,10006,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8219,24990,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8220,24992,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8221,24993,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
8222,24995,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [73]:
qual_model_2

,fwver_03.11.1149,fwver_03.11.1167,fwver_04.16.3345,fwver_04.16.3439,fwver_04.16.3553,fwver_04.16.3571,fwver_04.22.1442,fwver_04.22.1656,fwver_04.22.1666,fwver_04.22.1684,...,fwver_04.73.2571,fwver_04.82.1684,fwver_04.82.1778,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.2138,fwver_05.66.3237,fwver_05.66.3571,fwver_09.17.1431
user_id,,,,,,,,,,,,,,,,,,,,,
10000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
10004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10005,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10006,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24990,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24992,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24993,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [80]:
train_qual=pd.merge(qual_model_1,qual_model_2,how='left',on='user_id')

In [81]:
train_qual

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12,...,fwver_04.73.2571,fwver_04.82.1684,fwver_04.82.1778,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2122,fwver_05.15.2138,fwver_05.66.3237,fwver_05.66.3571,fwver_09.17.1431
0,10000,2.020113e+13,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,False,False,False,False,False,False,True,False,False,False
1,10002,2.020112e+13,0.000000,-0.020833,-0.010417,0.0,0.0,0.458333,-0.020833,0.0,...,False,False,False,False,False,False,True,False,False,False
2,10004,2.020110e+13,-0.083333,-0.083333,-0.083333,0.0,0.0,3.541667,-0.083333,0.0,...,False,False,False,False,False,False,False,False,False,False
3,10005,2.020112e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,1.083333,-0.416667,0.0,...,False,False,False,False,False,False,False,False,False,False
4,10006,2.020112e+13,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8219,24990,2.020111e+13,NaN,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0,...,False,False,False,False,False,False,False,False,False,False
8220,24992,2.020111e+13,-0.333333,-0.333333,-0.333333,0.0,0.0,-0.333333,-0.333333,0.0,...,False,False,False,False,False,False,False,False,False,False
8221,24993,2.020112e+13,-0.291667,-0.291667,-0.291667,0.0,0.0,-0.291667,-0.291667,0.0,...,False,False,False,False,False,False,True,False,False,False
8222,24995,2.020113e+13,-0.416667,-0.416667,-0.416667,0.0,0.0,-0.416667,-0.416667,0.0,...,False,False,False,False,False,False,False,False,False,False


### test_quality

In [82]:
ts_qual_model=test_quality

In [84]:
ts_qual_model

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,2,0,0,0,0,3,0,0
747968,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
747969,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
747970,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0


### (userid & fwv) 기준

In [85]:
ts_qual_model=ts_qual_model.groupby(['user_id','fwver']).mean()
ts_qual_model.reset_index(inplace=True)

In [86]:
ts_qual_model

,user_id,fwver,time,quality_0,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
0,30000,04.33.1261,2.020113e+13,0.000000,0.000000,0.0,0.0,0.416667,0.000000,0.0
1,30001,05.15.2138,2.020111e+13,-0.250000,-0.250000,0.0,0.0,-0.250000,-0.250000,0.0
2,30002,04.22.1750,2.020111e+13,-0.125000,-0.125000,0.0,0.0,-0.125000,-0.125000,0.0
3,30003,04.22.1750,2.020112e+13,0.000000,0.000000,0.0,0.0,4.444444,0.000000,0.0
4,30003,04.22.1778,2.020112e+13,0.000000,0.000000,0.0,0.0,2.500000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...
10054,44993,05.15.2138,2.020112e+13,0.145833,0.145833,0.0,0.0,2.229167,-0.041667,0.0
10055,44994,04.16.3553,2.020113e+13,-0.916667,-0.916667,0.0,0.0,-0.916667,-0.916667,0.0
10056,44996,04.22.1750,2.020111e+13,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0
10057,44996,04.22.1778,2.020112e+13,-0.104167,-0.104167,0.0,0.0,-0.104167,-0.104167,0.0


### fw 두개 이상 평균

In [90]:
ts_qual_model_1=ts_qual_model.groupby(['user_id']).mean()
ts_qual_model_1.reset_index(inplace=True)
ts_qual_model_2=pd.get_dummies(ts_qual_model[['user_id','fwver',]],columns=['fwver']).groupby(['user_id']).sum()
ts_qual_model_2=(ts_qual_model_2>0).reset_index()
test_qual=pd.merge(ts_qual_model_1,ts_qual_model_2,how='left',on='user_id')

In [91]:
ts_qual_model_1

,user_id,time,quality_0,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12
0,30000,2.020113e+13,0.000000,0.000000,0.0,0.0,0.416667,0.000000,0.0
1,30001,2.020111e+13,-0.250000,-0.250000,0.0,0.0,-0.250000,-0.250000,0.0
2,30002,2.020111e+13,-0.125000,-0.125000,0.0,0.0,-0.125000,-0.125000,0.0
3,30003,2.020112e+13,0.000000,0.000000,0.0,0.0,3.472222,0.000000,0.0
4,30004,2.020112e+13,-0.250000,-0.250000,0.0,0.0,1.125000,-0.250000,0.0
...,...,...,...,...,...,...,...,...,...
8214,44990,2.020111e+13,-0.083333,-0.083333,0.0,0.0,-0.083333,-0.083333,0.0
8215,44993,2.020112e+13,0.145833,0.145833,0.0,0.0,2.229167,-0.041667,0.0
8216,44994,2.020113e+13,-0.916667,-0.916667,0.0,0.0,-0.916667,-0.916667,0.0
8217,44996,2.020112e+13,-0.052083,-0.052083,0.0,0.0,-0.052083,-0.052083,0.0


In [92]:
test_qual

,user_id,time,quality_0,quality_2,quality_3,quality_4,quality_6,quality_11,quality_12,fwver_03.11.1149,...,fwver_04.73.2237,fwver_04.73.2571,fwver_04.82.1684,fwver_04.82.1778,fwver_05.15.2114,fwver_05.15.2120,fwver_05.15.2138,fwver_05.66.3237,fwver_05.66.3571,fwver_09.17.1431
0,30000,2.020113e+13,0.000000,0.000000,0.0,0.0,0.416667,0.000000,0.0,False,...,False,False,False,False,False,False,False,False,False,False
1,30001,2.020111e+13,-0.250000,-0.250000,0.0,0.0,-0.250000,-0.250000,0.0,False,...,False,False,False,False,False,False,True,False,False,False
2,30002,2.020111e+13,-0.125000,-0.125000,0.0,0.0,-0.125000,-0.125000,0.0,False,...,False,False,False,False,False,False,False,False,False,False
3,30003,2.020112e+13,0.000000,0.000000,0.0,0.0,3.472222,0.000000,0.0,False,...,False,False,False,False,False,False,False,False,False,False
4,30004,2.020112e+13,-0.250000,-0.250000,0.0,0.0,1.125000,-0.250000,0.0,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8214,44990,2.020111e+13,-0.083333,-0.083333,0.0,0.0,-0.083333,-0.083333,0.0,False,...,False,False,False,False,False,False,False,False,False,False
8215,44993,2.020112e+13,0.145833,0.145833,0.0,0.0,2.229167,-0.041667,0.0,False,...,False,False,False,False,False,False,True,False,False,False
8216,44994,2.020113e+13,-0.916667,-0.916667,0.0,0.0,-0.916667,-0.916667,0.0,False,...,False,False,False,False,False,False,False,False,False,False
8217,44996,2.020112e+13,-0.052083,-0.052083,0.0,0.0,-0.052083,-0.052083,0.0,False,...,False,False,False,False,False,False,False,False,False,False


<br>

### fwver

In [12]:
# train
# fwver_col=pd.get_dummies(err[['user_id','fwver']],columns=['fwver']).groupby(['user_id']).sum()
# fwver_col=(fwver_col>0).reset_index()
# train=pd.merge(train,fwver_col)

#test
test_err.loc[~test_err['fwver'].isin(err['fwver'].unique()),'fwver']=np.nan
fwver_col=pd.get_dummies(test_err[['user_id','fwver']],columns=['fwver']).groupby(['user_id']).sum()
fwver_col=(fwver_col>0).reset_index()
test=pd.merge(test,fwver_col,how='left',on='user_id')

<br>

### errtype

In [93]:
# train
errtype_col=pd.get_dummies(err[['user_id','errtype']],columns=['errtype']).groupby(['user_id']).sum()
errtype_col=(errtype_col>0).reset_index()
train=pd.merge(train,errtype_col)

# test
test_err.loc[~test_err['errtype'].isin(err['errtype'].unique()),'errtype']=np.nan
errtype_col=pd.get_dummies(test_err[['user_id','errtype']],columns=['errtype']).groupby(['user_id']).sum()
errtype_col=(errtype_col>0).reset_index()
test=pd.merge(test,errtype_col,how='left',on='user_id')

MemoryError: Unable to allocate 5.06 GiB for an array with shape (16554663, 41) and data type float64

<br>

### errcode

In [14]:
# train
err_val=err['errcode'].value_counts(normalize=True).cumsum()[:20].index
err.loc[~(err['errcode'].isin(err_val).values),'errcode']='etc'

In [15]:
# test
test_err.loc[~(test_err['errcode'].isin(err_val).values),'errcode']='etc'

In [16]:
# train
errcode_col=pd.get_dummies(err[['user_id','errcode']],columns=['errcode']).groupby(['user_id']).sum()
errcode_col=(errcode_col>0).reset_index()
train=pd.merge(train,errcode_col)

# test
test_err.loc[~test_err['errcode'].isin(err['errcode'].unique()),'errcode']=np.nan
errcode_col=pd.get_dummies(test_err[['user_id','errcode']],columns=['errcode']).groupby(['user_id']).sum()
errcode_col=(errcode_col>0).reset_index()
test=pd.merge(test,errcode_col,how='left',on='user_id')

<br>

### problem

In [17]:
prob_col=np.array(np.zeros(15000))
prob_col[problem['user_id'].unique()-10000]=1

In [18]:
prob_col

array([0., 1., 0., ..., 1., 1., 0.])

<br>

### 컬럼 정리

In [19]:
# 없는 행이 하나 존재
test.isna().sum()

user_id                                 0
model_nm_model_0                        1
model_nm_model_1                        1
model_nm_model_2                        1
model_nm_model_3                        1
                                       ..
errcode_active                          1
errcode_connection fail to establish    1
errcode_connection timeout              1
errcode_etc                             1
errcode_standby                         1
Length: 103, dtype: int64

In [20]:
# nan값은 False로 채워 넣는다
test=test.fillna(False)

In [21]:
# test set에 없는 컬럼은 False로 설정
for col in train.columns:
    if not col in test.columns:
        print(col)
        test[col]=False

In [22]:
# 순서 맞추기
test=test[train.columns]

<br><br>

## Modeling

In [31]:
X=train.loc[:,'model_nm_model_0':'errcode_standby']
y=prob_col
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=2020)
print(X_train.shape)                                                                                                                                                                                                                                                                                                                                                                                                                                                           7
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(11250, 108)
(3750, 108)
(11250,)
(3750,)


In [32]:
X_train

,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,...,errcode_94,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_etc,errcode_standby
11512,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
1565,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,True
3925,True,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,False,False,True,True,False
7524,False,False,True,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,True,True,True
811,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11971,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
14966,False,True,False,False,False,False,False,False,False,False,...,False,False,True,True,True,False,True,True,True,False
7491,False,False,True,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,True,True,True
12680,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,False,True,False,False,True,True


In [52]:
light_gbm=lgb.Dataset(X_train, label=y_train)

In [234]:
params={'learning_rate':0.01,'boosting_type':'dart','objective':'binary','metric':'binary_logloss','num_leaves': 40,
        'max_depth':30,'feature_fraction': 0.6,'subsample': 0.7}
clf = lgb.train(params, light_gbm, 4000)

[LightGBM] [Info] Number of positive: 5000, number of negative: 10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005483 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 172
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 86
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147


In [235]:
y_pred=clf.predict(X_train)
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0
accuracy_score(y_pred,y_train)

0.8206222222222223

In [236]:
y_pred=clf.predict(X_test)
y_pred[y_pred>0.5]=1
y_pred[y_pred<=0.5]=0

In [237]:
sum(y_pred==1)

762

In [238]:
accuracy_score(y_pred,y_test)

0.8216

<br>

## Submission

In [40]:
test

,user_id,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,...,errcode_94,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_etc,errcode_standby
0,30000,False,True,True,False,False,False,False,False,False,...,False,False,True,False,True,True,True,True,True,True
1,30001,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,30002,True,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,True,True,True
3,30003,True,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,True,True,False
4,30004,False,True,True,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,False,True,False,False,False,False,False,False,False,...,False,False,False,True,True,False,True,True,True,False
14995,44995,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
14996,44996,True,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,True,True,True,True
14997,44997,True,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,True,True,True,True


In [41]:
test.loc[:,'model_nm_model_0':'errcode_standby']

,model_nm_model_0,model_nm_model_1,model_nm_model_2,model_nm_model_3,model_nm_model_4,model_nm_model_5,model_nm_model_6,model_nm_model_7,model_nm_model_8,fwver_03.11.1141,...,errcode_94,errcode_95,errcode_B-A8002,errcode_NFANDROID2,errcode_S-61001,errcode_active,errcode_connection fail to establish,errcode_connection timeout,errcode_etc,errcode_standby
0,False,True,True,False,False,False,False,False,False,False,...,False,False,True,False,True,True,True,True,True,True
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,True,True,True
3,True,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,False,True,True,False
4,False,True,True,False,False,False,False,False,False,False,...,False,False,False,True,True,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,False,True,False,False,False,False,False,False,False,False,...,False,False,False,True,True,False,True,True,True,False
14995,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,True,False
14996,True,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,True,True,True,True,True
14997,True,False,False,False,False,False,False,False,False,False,...,False,False,True,True,False,True,True,True,True,True


In [241]:
light_gbm=lgb.Dataset(X, label=y)

In [242]:
clf = lgb.train(params, light_gbm, 4000)

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 5000, number of negative: 10000
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002468 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 172
[LightGBM] [Info] Number of data points in the train set: 15000, number of used features: 86
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.333333 -> initscore=-0.693147
[LightGBM] [Info] Start training from score -0.693147


In [243]:
y_pred=clf.predict(test.loc[:,'model_nm_model_0':'errcode_standby'])

In [244]:
submission=pd.read_csv('./data/sample_submission.csv')

In [245]:
submission['problem']=y_pred

In [246]:
submission.to_csv('./result/submission ver1.0.2.csv',index=False)